In [1]:
import sys
sys.path.append('..')
import torch
import numpy as np
import torch.nn as nn
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
import torchvision.transforms as tf

from src.models.hrnet.model import HRNet
from src.models.train import Trainer
from src.features.segmentation.dataset import SegmentationDataset

In [2]:
hrnet = HRNet()
#swin = Swin()

In [3]:
for name, param in hrnet.named_parameters():
    if param.requires_grad:
        print(name) 

net.cls_head.weight
net.cls_head.bias
net.aux_head.0.weight
net.aux_head.0.bias
net.aux_head.1.weight
net.aux_head.1.bias
net.aux_head.3.weight
net.aux_head.3.bias


In [10]:
trainer = Trainer(
    model=hrnet,
    device="mps",
    optimizer=torch.optim.Adam(hrnet.parameters(), lr=10e-5)
)

In [11]:
transform = tf.Resize((256,256))

In [12]:
train_dataset = SegmentationDataset(
    images_dir="../data/split/train/train_images/",
    masks_dir="../data/split/train/train_masks/",
    image_transform=transform,
    mask_transform=transform
)

val_dataset = SegmentationDataset(
    images_dir="../data/split/valid/valid_images/",
    masks_dir="../data/split/valid/valid_masks/",
    image_transform=transform,
    mask_transform=transform
)


train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=2, shuffle=False)


In [ ]:
adele_dataset = SegmentationDataset(
    images_dir="../data/split/train/train_images/",
    masks_dir="../data/split/train/train_masks/",
    image_transform=transform,
    mask_transform=transform,
    use_adele=True,
    return_names=True
)

In [19]:
trainer.train(
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader
)

  2%|▏         | 127/6807 [01:15<1:05:54,  1.69it/s]


KeyboardInterrupt: 